In [1]:
import pandas as pd
import numpy as np
from tPreprocessing import preprocess_dataset
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
RANDOM_STATE = 41

In [2]:
columns = ["Pclass", "Age", "FamilySize", "Fare", "CabinsAmount","Sex","NamePrefix", "TicketPrefix", "CabinDeck", "Embarked"]

X_train, X_final_test, y, columnNames = preprocess_dataset(columns, dropColumn="first", degree=3)
# Test set from Kaggle does not have prediction result, so I should split training set
X_tmp_train, X_tmp_test, y_tmp_train, y_tmp_test = train_test_split(X_train, y, random_state = RANDOM_STATE)
print(columnNames)

Deleted columns:  ['x1', 'x3', 'x0^2', 'x0 x1', 'x0 x3', 'x0 x4', 'x1^2', 'x1 x2', 'x2 x3', 'x3 x4', 'x0^3', 'x0^2 x1', 'x0^2 x3', 'x0 x1^2', 'x0 x1 x2', 'x0 x2^2', 'x0 x2 x3', 'x0 x2 x4', 'x0 x3^2', 'x0 x4^2', 'x1^2 x2', 'x1^2 x3', 'x1 x2^2', 'x1 x2 x3', 'x1 x3^2', 'x1 x3 x4', 'x1 x4^2', 'x2^3', 'x2^2 x3', 'x2^2 x4', 'x2 x3^2', 'x2 x3 x4', 'x3^3', 'x3^2 x4', 'x4^3', 'x0_male', 'x1_Dr', 'x1_Jonkheer', 'x1_Mme', 'x1_Mrs', 'x1_Rev']
['1', 'x0', 'x2', 'x4', 'x0 x2', 'x1 x3', 'x1 x4', 'x2^2', 'x2 x4', 'x3^2', 'x4^2', 'x0^2 x2', 'x0^2 x4', 'x0 x1 x3', 'x0 x1 x4', 'x0 x3 x4', 'x1^3', 'x1^2 x4', 'x1 x2 x4', 'x2 x4^2', 'x3 x4^2', 'x1_Col', 'x1_Don', 'x1_Dona', 'x1_Lady', 'x1_Major', 'x1_Master', 'x1_Miss', 'x1_Mlle', 'x1_Mr', 'x1_Ms', 'x1_Sir', 'x1_th', 'x2_A/5', 'x2_A/S', 'x2_A2', 'x2_A4', 'x2_A5', 'x2_AQ/3', 'x2_AQ/4', 'x2_C', 'x2_CA', 'x2_CA/SOTON', 'x2_FC', 'x2_FCC', 'x2_Fa', 'x2_LP', 'x2_N/A', 'x2_P/PP', 'x2_PC', 'x2_PP', 'x2_SC', 'x2_SC/A3', 'x2_SC/A4', 'x2_SC/AH', 'x2_SC/AHBasle', 'x2_S

In [3]:
param_grid={ 'C': [0.001, 0.005, 0.007, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.1, 0.5, 1, 5, 7, 8, 9, 10],
'class_weight':['balanced', None],
'penalty': ["l1", "l2"]}
cvList = [None,2,3,4,5,6,7]
for cv in cvList:
    optimizer = GridSearchCV(LogisticRegression(fit_intercept=True,random_state = RANDOM_STATE), param_grid, cv=cv, n_jobs=-1)
    optimizer.fit(X_tmp_train, y_tmp_train)
    scoreTrain = optimizer.score(X_tmp_train, y_tmp_train)
    scoreTest = optimizer.score(X_tmp_test, y_tmp_test)
    print("CV ", cv, "Lambda ", optimizer.best_estimator_.C, "Train score ", scoreTrain, "Test score ", scoreTest, "Mean ", (scoreTest+scoreTrain)/2)
    print( "Hyperparameters: ", optimizer.best_estimator_, "Parameters: ", optimizer.best_params_)
    # roc_auc_score(y_tmp_test, optimizer.predict(X_tmp_test))

CV  None Lambda  0.1 Train score  0.812874251497006 Test score  0.8385650224215246 Mean  0.8257196369592653
Hyperparameters:  LogisticRegression(C=0.1, random_state=41) Parameters:  {'C': 0.1, 'class_weight': None, 'penalty': 'l2'}
CV  2 Lambda  0.06 Train score  0.8158682634730539 Test score  0.8385650224215246 Mean  0.8272166429472892
Hyperparameters:  LogisticRegression(C=0.06, random_state=41) Parameters:  {'C': 0.06, 'class_weight': None, 'penalty': 'l2'}
CV  3 Lambda  0.1 Train score  0.812874251497006 Test score  0.8385650224215246 Mean  0.8257196369592653
Hyperparameters:  LogisticRegression(C=0.1, random_state=41) Parameters:  {'C': 0.1, 'class_weight': None, 'penalty': 'l2'}
CV  4 Lambda  0.1 Train score  0.812874251497006 Test score  0.8385650224215246 Mean  0.8257196369592653
Hyperparameters:  LogisticRegression(C=0.1, random_state=41) Parameters:  {'C': 0.1, 'class_weight': None, 'penalty': 'l2'}
CV  5 Lambda  0.1 Train score  0.812874251497006 Test score  0.83856502242152

In [4]:
## Write answer
answer = pd.DataFrame(columns=["PassengerId", "Survived"])
dfTest = pd.read_csv("./test.csv")
answer["PassengerId"] = dfTest["PassengerId"]
answer["Survived"] = optimizer.predict(X_final_test)
answer.to_csv("answer.csv", index=False)